In [387]:
!ls

README.md	     data_utils.py  new_trade	      sweep.yml
__pycache__	     dec_trade	    old_trade	      train.py
base_config.yml      eda.py	    parser_maker.py   train_copy.py
chan_cur	     etc	    predictions       train_inference.py
chan_wd		     eval_utils.py  prepare.py	      train_loop.py
change_ont_value.py  evaluation.py  preprocessor      training_recorder.py
conf.yml	     graph	    requirements.txt  wandb
conf2.yml	     inference.py   results	      wandb_stuff.py
conf3.yml	     losses.py	    small_trade       whole-in-one.py
configs		     model	    submit.py


In [1]:
%cd ~/project/team/code

/opt/ml/project/team/code


In [2]:
from data_utils import (WOSDataset, load_dataset,
                        seed_everything)
from evaluation import _evaluation

from train_loop import trade_train_loop, submt_train_loop
from inference import trade_inference, sumbt_inference 

from prepare import get_data, get_stuff, get_model

from attrdict import AttrDict

In [55]:
args = AttrDict(
    data_dir= '/opt/ml/input/data/train_dataset',
    ontology_root='/opt/ml/input/data/train_dataset/edit_ontology_metro.json',
    use_domain_slot='basic',
    
    use_small_data=False,
    train_batch_size=8,
    model_name_or_path= 'dsksd/bert-ko-small-minimal',
    vocab_size=35000,
    preprocessor='TRADEPreprocessor',
    model_class='TRADE',
    ModelName='TRADE',
    use_convert_ont=False,
    use_sys_usr_sys_turn=False,
)

model_folder = 'results/decoder_long/'
model_name = 'model-best.bin'

In [56]:
data, slot_meta, ontology = get_data(args)
train_data, dev_data, _, _ = load_dataset(data, 0)

## 둘 중 하나 고르세요.

In [57]:
def filter_by_domain(data, domain):
    found_idx = set()
    for i in range(len(data)):
        if domain in data[i]['domains']:
            found_idx.add(i)
    return [data[i] for i in found_idx]

domain = '지하철'
small_data = filter_by_domain(train_data, domain)

In [60]:
def filter_by_value(data, target):
    found_idx = set()
    for i in range(len(data)):
        find_next = False
        for dig in data[i]['dialogue']:
            if 'state' in dig:
                for st in dig['state']:
                    domain, slot, value =  st.split('-')
                    if value == target:
                        find_next = True
                        found_idx.add(i)
                        break
            if find_next:
                break
    return [data[i] for i in found_idx]

target = '강남역'
small_data = filter_by_value(train_data, target)

In [61]:
tokenizer, processor, train_features, dev_features = get_stuff(args,
                 small_data, None, slot_meta, ontology)

Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors


In [22]:
import json
import argparse
import torch
from torch.utils.data import DataLoader, SequentialSampler



tokenized_slot_meta = []
for slot in slot_meta:
    tokenized_slot_meta.append(
        tokenizer.encode(slot.replace("-", " "), add_special_tokens=False)
    )
config = json.load(open(f"{model_folder}/exp_config.json", "r"))
config = argparse.Namespace(**config)
model =  get_model(config, tokenizer, ontology, slot_meta)
ckpt = torch.load(f'{model_folder}/{model_name}', map_location="cpu")
model.load_state_dict(ckpt)

train_data2 = WOSDataset(train_features)
train_sampler = SequentialSampler(train_data2)
train_loader = DataLoader(
    train_data2,
    batch_size=args.train_batch_size,
    sampler=train_sampler,
    collate_fn=processor.collate_fn,
)

## 일단 TRADE 모델 꺼

In [23]:
def postprocess_state(state):
    for i, s in enumerate(state):
        s = s.replace(" : ", ":")
        s = s.replace(" & ", "&")
        s = s.replace(" = ", "=")
        s = s.replace("( ", "(")
        s = s.replace(" )", ")")
        state[i] = s.replace(" , ", ", ")
    return state

In [64]:
device = torch.device('cpu') # 개인 사정에 맞게 (cpu | cuda)
model.to(device)
print()

In [ ]:
from tqdm.auto import tqdm
from training_recorder import RunningLossRecorder
from torch.cuda.amp import autocast
use_amp = False

model.eval()
predictions = {}
no_post = {}
eval_loader = train_loader
pbar = tqdm(eval_loader, total=len(eval_loader))
loss_recorder = RunningLossRecorder(len(eval_loader))
cnt = 0
for batch in pbar:
    input_ids, segment_ids, input_masks, gating_ids, target_ids, guids = [
        b.to(torch.device(device)) if not isinstance(b, list) else b for b in batch
    ]

    with torch.no_grad():
        with autocast(enabled=use_amp):
            o, g = model(input_ids, segment_ids, input_masks, 9)

        _, generated_ids = o.max(-1)
        _, gated_ids = g.max(-1)


    for guid, gate, gen in zip(guids, gated_ids.tolist(), generated_ids.tolist()):
        prediction = processor.recover_state(gate, gen)
        no_post[guid] = prediction
        prediction = postprocess_state(prediction)
        predictions[guid] = prediction

    cnt += 1
#     if cnt == 1:
#         break
    
pbar.close()


In [33]:
from data_utils import get_examples_from_dialogues
user_first = False
dialogue_level = False
processor_kwargs = AttrDict()
    
train_examples = get_examples_from_dialogues(
    small_data, user_first=user_first, dialogue_level=dialogue_level,use_sys_usr_sys=args.use_sys_usr_sys_turn,
        which='train'
)

## 보는 방법은 CONTROL + ENTER로

* MISSED: 정답인데 예측 하지 않은 거
*  WRONG: 예측한 값이 틀린 경우
* CORREC: 제대로 예측한 경우

In [45]:
idx = 0
ln = len(train_examples)

In [67]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
while idx < ln:
    datum = train_examples[idx]
    idx += 1
    guid = datum.guid
    answer = set(datum.label)
    pred = set(predictions[guid])
    if set(datum.label) != set(predictions[guid]):
        print(f'dialogue {idx}')
        for i in range(len(datum.context_turns)//2):
            print(f'SYS: {datum.context_turns[2*i]}')
            print(f'USR: {datum.context_turns[2*i + 1]}')

        print()
        print(f'SYS: {datum.current_turn[0]}')
        print(f'USR: {datum.current_turn[1]}')
        if args.use_sys_usr_sys_turn:
            print(f'SYS: {datum.current_turn[2]}')
        print()
        tab = '\t'
        print(f'MISSED: {tab.join(sorted(answer - pred))}')
        print(f'WRONG : {tab.join(sorted(pred - answer))}')
        print(f'CORREC: ')
        pp.pprint(sorted(answer & pred))
        break
 

dialogue 30
SYS: 
USR: 서울 남쪽으로 비싼 가격대의 식당을 찾고있어요. 
SYS: 안녕하세요. 식당의 종류는 어떻게 도와드릴까요? 
USR: 종류는 아무거나 괜찮아요. 
SYS: 팁 앤 스테이크 어떠세요? 평점은 3.8 입니다. 
USR: 대표 메뉴 알려주세요. 예약도 해주시고요. 토요일에 20시 3명이요. 
SYS: 예약 번호는 OIFM2 입니다. 대표 메뉴는 스테이크입니다. 
USR: 아 그리고 가까운 지하철역 알려주세요. 
SYS: 강남역으로 나옵니다. 
USR: 감사합니다. 숙소도 알아봐주시겠어요? 조식 제공되는 적당한 가격대의 숙소로요. 

SYS: 숙소의 종류와 지역은 어떻게 할까요? 
USR: 크게 상관없으니 알아서 해주세요. 가까운 지하철역이 어디인지도 같이 알아봐주세요. 
SYS: 김의 옥탑방 어떠세요? 평점은 4.5 입니다. 가까운 지하철역은 노원역입니다. 

MISSED: 
WRONG : 숙소-이름-김의 옥탑방
CORREC: 
[   '숙소-가격대-적당',
    '숙소-조식 가능-yes',
    '숙소-종류-dontcare',
    '숙소-지역-dontcare',
    '식당-가격대-비싼',
    '식당-예약 명수-3',
    '식당-예약 시간-20:00',
    '식당-예약 요일-토요일',
    '식당-이름-팁 앤 스테이크',
    '식당-종류-dontcare',
    '식당-지역-서울 남쪽']
